# **ARBOLES DE DECISIÓN**
---
**Integrantes:**
- Condori Lopez, Juan Carlos
- Espejo Franco, Melissa Brigitte
- Monzon Montalvo, Alexander Junior
- Ramos Alvarez, Edgar
- Rojas Soto, Claudia Luz
- Salcedo Hurtado, Jorge Andre

El árbol de decisiones es uno de los algoritmos más fáciles de visualizar e interpretar, lo que resulta útil cuando se presentan resultados a una audiencia no técnica, como suele ser necesario en la industria. Si simplemente consideramos un árbol en un estado similar a un diagrama de flujo, desde la raíz hasta las hojas, donde el camino a una hoja desde la raíz define las reglas de decisión sobre las características, entonces ya tenemos un buen nivel de intuición necesario para comprender el aprendizaje del árbol de decisión.

## **1. Árboles de clasificación y regresión**
---
CART (Classification and Regression Trees) inicializa la raíz del árbol con el dataset de entranemiento completo (ver Figura). Luego busca dividir los datos del dataset con una pregunta. Los nodos siguientes reciben solamente los datos que corresponde a la respuesta. Se reitera este proceso hasta desenredar totalmente los datos (cada hoja del árbol debería tener idealmente un solo tipo de etiqueta).

<center/>
<img href="https://ibb.co/rfppRTg"><img src="https://i.ibb.co/vvdd6gb/Diagrama-en-blanco-9.png" alt="Diagrama-en-blanco-9" border="0" width=500px/>
</center>

En nuestro ejemplo, el nodo 1 está totalmente desenredado (Etiqueta: "Uva"). El nodo 2 tiene dos labeles, entonces preguntamos otra pregunta:

<center/>
<a href="https://ibb.co/jbHtRzH"><img src="https://i.ibb.co/34kKzvk/Diagrama-en-blanco-P-gina-1-2.png" alt="Diagrama-en-blanco-P-gina-1-2" border="0"  width=500px></a>
</center>

Para construir un árbol eficiente, el punto importante es identificar qué preguntas formular y cuándo. Por lo tanto, necesitamos cuantificar en qué medida una pregunta permite desenredar las etiquetas. Para hacer eso se utiliza 2 métricas:

- El coeficiente de 'Gini impurity': mide que tan desenredados están los labeles de un nodo.
- El coeficiente de 'Information Gain': mide cuánto una pregunta permite bajar el 'Gini impurity'.

<center/>
<a href="https://ibb.co/gZFQcYx"><img src="https://i.ibb.co/TMqJNys/3.png" alt="3" border="0" width=500px></a>
</center>

In [ ]:
!pip install pyspark

In [ ]:
from pyspark import SparkContext
sc =SparkContext()
import os

In [ ]:
# Librería utilizada para paralelizar procesos utilizando los nucleos del procesador
from multiprocessing.pool import ThreadPool 


### **Valores únicos**
---

In [ ]:
def unique_vals(rows, col):
    """Encuentre los valores únicos para una columna en un conjunto de datos.
    Args:
        rows (list of lists): Dataset
        col (int): Indica la columna con la que se está trabajando
    Returns:
        list: Valores únicos la columna evaluada
    """
    # Creamos la rdd
    sparkRow=sc.parallelize(rows,os.cpu_count())
    # Mapeamos el rdd para la columna evaluada
    sparkColumn=sparkRow.map(lambda x:x[col])
    # Reducimos extrayendo los valores únicos de la columna
    unique_vals=sparkColumn.map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y).map(lambda x:x[0])
    # Devolvemos nuestra lista de valores únicos
    return sorted(unique_vals.collect())

### **Contador de clases**
---

In [ ]:
import functools
import itertools
def class_counts(rows):
    """Cuenta el número de cada tipo de ejemplo en un conjunto de datos.
    Args:
        Rows(list of list): Dataset
    Returns:
        Dict(devuelve la labels de las clases y la cantidad de repeticiones en el dataset)
    """
    #Mapeamos la columna final que contiene las clases del dataset
    LastColumn=map(lambda x:x[-1],rows)
    #Agregamos un valor de 1 a cada valor para la reduccion
    Clases=map(lambda x:(x,1),LastColumn)
    list_key=[]
    list_values=[]
    #Reducimos nuestros datos clase
    for key, group in itertools.groupby(list(Clases), lambda x: x[0]):
        list_key.append(key)#guardamos nuestra clase
        list_values.append(sum(list(map(lambda x:x[1],list(group)))))#guardamos las repeticiones de las clases
    dict_from_list = dict(zip(list_key,list_values))#Creamos nuestro diccionario
    return dict_from_list

### **Verificador númerico**
---

In [ ]:
def is_numeric(value):
    """Prueba si un valor es numérico.
    Args:
        value (any): Cualquier valor
    Returns:
        boolean: retorna True si es un valor numérico
    """
    return isinstance(value, int) or isinstance(value, float)

## **2. ¿Qué preguntas formular?**
---

Para saber qué preguntas formular, cada nodo itera sobre las características de los datos a su disposición y define una lista de preguntas posibles.
<center/>
<a href="https://ibb.co/VC4BSJs"><img src="https://i.ibb.co/QHwmNp4/2.png" alt="2" border="0" width=500px></a>
</center>


### **Clase Pregunta:**
---
Una pregunta se utiliza para particionar un conjunto de datos.Esta clase solo registra un 'número de columna' (por ejemplo,0 para Color) y un 'valor de columna' (por ejemplo, Verde). El método 'match' se utiliza para comparar el valor de característica en un ejemplo al valor de característica almacenado en la pregunta.

In [ ]:
class Question:
    """
    Atributos:
        column(int): indica el numero de columna
        value(str):indica el valor de la columna
    """
    def __init__(self, column, value):
        self.column = column
        self.value = value
    
    def match(self, example):
        """Compara el valor de característica en un ejemplo al valor de característica 
            almacenado en la pregunta.
            Args:
                example(list):una fila que contiene una muestra del dataset 
            Returns:
                boolean(bool):indica si el valor es mayor o igual si es numerico
                            o igual si es categorico
        """
        # Compara el valor de la función en un ejemplo con el
        # valor de característica en esta pregunta.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        """
        Método auxiliar para imprimir la pregunta en un formato legible.
            Returns(str): ">=" en caso sea numerico
                          "sentencia" en caso sea categorico
        """
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Es %s %s %s?" % (
            header[self.column], condition, str(self.value))

## **3. Partición**
---
Una vez una pregunta elegida, se divide los datos en dos según la respuesta a la pregunta.

<center/>
<a href="https://ibb.co/9rcNBgP"><img src="https://i.ibb.co/x7XDwLZ/1.png" alt="1" border="0" width=500px></a>
</center>

### **Partición de los datos**
---

In [ ]:
def partition(rows, question):
    """Particiona un conjunto de datos.
    Para cada fila del conjunto de datos, se comprueba si coincide 
    con la pregunta. Si entonces, se agrega a 'filas verdaderas',
    de lo contrario, se agrega a 'filas falsas'.

    Args:
        rows (list of lists): Dataset
        question (object): Validador
    Returns:
        list: Valores únicos la columna evaluada
    """
    verdaderas = list(filter(lambda x: question.match(x) == True,rows))
    falsas = list(filter(lambda x: question.match(x) == False,rows))
    return verdaderas, falsas


## **4. Coeficiente de Gini**
---
El coeficiente de Gini impurity representa la probabilidad de ser incorrecto si asigna aleatoriamente una etiqueta a un ejemplo del mismo conjunto. Por ejemplo, en los dos ejemplos siguientes: ¿Cuál es la probabilidad de equivocarse si asignamos una etiqueta del recipiente B a un dato del recipiente A?

<center/>
<a href="https://ibb.co/jr90mS0"><img src="https://i.ibb.co/kSVbjtb/Diagrama-en-blanco-P-gina-1-3.png" alt="Diagrama-en-blanco-P-gina-1-3" border="0" width=500px></a>
</center>


### **Impureza de Gini**
---

La impureza Gini es una medida que se utiliza para generar árboles de clasificación. Proporciona más información acerca de la distribución de los datos por nodo que la precisión de clasificación utilizada para informes de precisión del árbol.

La impureza de un nodo del árbol de clasificación se calcula utilizando el recuento de cada categoría de destino entre todos los registros correspondientes para un nodo dado. El total de impureza Gini se calcula como una suma de cuadrados del recuento de proporciones entre todas las categorías de destino por nodo al que se resta uno y los resultados se multiplican por el número de registros.

Por ejemplo, cuando se divide un nodo de árbol, el algoritmo busca un campo con la mejora más elevada del total de impureza calculada como el total de impureza entre todos los nodos hijo potenciales restados del total de impureza del nodo padre.

Fuente: https://www.ibm.com/docs/es/cognos-analytics/11.1.0?topic=terms-gini-impurity-measure

Para calcular la impureza de Gini de un conjunto de elementos, supongamos $i$ toma valores en $\{ 1 , 2 , . . . , m \}$, y sea $f_i$ la fracción de artículos etiquetados con valor $i$ en el conjunto.
$$
I_G(f)=\sum_{i=1}^{m}f_i(1-f_i) = \sum_{i=1}^{m}(f_i-f_i^2)=\sum_{i=1}^{m}f_i-\sum_{i=1}^{m}f_i^2=1-\sum_{i=1}^{m}f_i^2
$$

### Intuición del índice de Gini:
Comencemos con Gini Index, ya que es un poco más fácil de entender. Según Wikipedia , el objetivo es "medir la frecuencia con la que un elemento elegido al azar del conjunto sería etiquetado incorrectamente".

Para visualizar esto, volvamos a los ejemplos de Gumball. Si decidimos etiquetar arbitrariamente los 4 chicles como rojos, ¿con qué frecuencia se etiquetaría incorrectamente uno de los chicles?

### 4 rojo y 0 azul:
$$
GiniIndex = 1-(probabilidadRojo^2+probabilidadAzul^2) = 1-(1^2+0^2) =0
$$
La medida de impureza es 0 porque nunca etiquetaríamos incorrectamente ninguno de los 4 chicles rojos aquí. Si elegimos arbitrariamente etiquetar todas las bolas como 'azules', entonces nuestro índice aún sería 0, porque siempre etiquetaríamos incorrectamente las bolas de chicle.

El puntaje de gini es siempre el mismo sin importar qué clase arbitraria tome las probabilidades porque siempre se suman a 0 en la fórmula anterior.

```
Una puntuación gini de 0 es la puntuación más pura posible.
```

### 2 rojos y 2 azules:
$$ GiniIndex = 1-(probabilidadRojo^2+probabilidadAzul^2)=1-(0.5^2+0.5^2)=0.5 $$

La medida de impureza es 0.5 porque etiquetaríamos incorrectamente las bolas de chicle como la mitad del tiempo. Debido a que este índice se utiliza en variables de destino binarias (0,1), un índice de Gini de 0,5 es la puntuación menos pura posible. La mitad es un tipo y la mitad es el otro. **Dividir las puntuaciones de gini por 0,5 puede ayudar a comprender intuitivamente lo que representa la puntuación. 0.5 / 0.5 = 1, lo que significa que la agrupación es lo más impura posible (en un grupo con solo 2 resultados).**

In [ ]:
import numpy as np
def gini(filas):
    """Calcula la impureza de Gini para una lista de filas.
    Args:
        filas (list of lists): Dataset.
    Returns:
        float: Valor de la impureza de Gini obtenido.

    """
    cantidadXClase = class_counts(filas)
    nroFilas = float(len(filas))
    impurezaGini = 1
    gini = map((lambda lbl: np.power(cantidadXClase[lbl]/nroFilas,2)),cantidadXClase)
    res = sum(list(gini))
    impurezaGini-=res
    return impurezaGini

## **5. Information gain**
---
La métrica de Information Gain permite medir qué pregunta optimiza el coeficiente de Gini impurity.
Por cada nodo, empezamos por medir el coeficiente de Gini impurity de las etiquetas disponibles. Luego, por cada pregunta calculamos el coeficiente de Gini impurity de los dos sub-conjuntos de datos obtenidos. 

Para un nodo S en el conjunto de datos con c clases objetivo, el índice Gini se define como:

$$Gini(S) = 1-\sum_{i=1}^c(p_i)^2$$
Teniendo a  $p_i$ como la proporción de $S$ que pertenece a la clase $i$. El cálculo del índice de Gini de una división con el atributo $A$ se define como:
$$
Gini(A_{split}) = \sum_{i=1}^c p_i Gini(F_i)
$$
Donde $p_i$ es la proporción de la clase $i$ después de la división. $F$ representa las características. El cálculo de la ganancia de información sobre el atributo $A$ después de la división se define como:
$$
Gain (S, A) = Gini(S) - Gini(A_{split})
$$

Fuente: Mohammad A. (2018) A Comprehensive Study of Decision Trees to Classify DNA Sequences. Universidad de Nuevo México.


<center/>
<a href="https://ibb.co/JrDt8yH"><img src="https://i.ibb.co/QYyQhCM/Diagrama-en-blanco-P-gina-1-4.png" alt="Diagrama-en-blanco-P-gina-1-4" border="0" width=500px></a>
</center>

<center/>
<a href="https://ibb.co/JrDt8yH"><img src="https://sitiobigdata.com/wp-content/uploads/2019/09/1569525641_392_Conceptos-b%C3%A1sicos-de-aprendizaje-autom%C3%A1tico-%C3%81rbol-de-decisi%C3%B3n-desde-cero-Parte-I.png" alt="Diagrama-en-blanco-P-gina-1-4" border="0" width=500px></a>
</center>

<center/>
<a href="https://ibb.co/JrDt8yH"><img src="https://sitiobigdata.com/wp-content/uploads/2019/09/1569525641_183_Conceptos-b%C3%A1sicos-de-aprendizaje-autom%C3%A1tico-%C3%81rbol-de-decisi%C3%B3n-desde-cero-Parte-I.png" alt="Diagrama-en-blanco-P-gina-1-4" border="0" width=500px></a>
</center>



In [ ]:
def info_gain(left, right, current_uncertainty):
    """Ganancia de información.
     La incertidumbre del nodo inicial, menos la impureza ponderada de
     dos nodos secundarios.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

### **Encontrar la mejor división**
---

In [ ]:
def find_best_split(rows):
    """Encuentra la mejor pregunta para hacer iterando sobre cada función/valor y calculando 
    la ganancia de información.

    Args:
        rows (list): Dataset
    Returns:
        best_gain: Mejor ganancia de información
        best_question: Mejor pregunta con mejor ganancia
    """
    # Recuperamos la incertidumbre dini de los datos
    current_uncertainty = gini(rows)
    # Recuperamos el numero de columnas de la data
    n_features = len(rows[0]) - 1 
    
    result = []
    # Recorremos cada columna de la data
    for col in range(n_features):

        def best_question_and_gain(val, col = col, rows = rows):
            """Encuentra la mejor pregunta calculando la ganancia de información.

            Args:
                val (): Valor a evaluar
                col (int): Valor de la columna
                rows (int): Valor de la fila
            Returns:
                col: Columna en la cual se trabajo
                val: Valor el cual se evaluo
                gain: Ganancia obtenida
            """

            # Creamos un clase Question con los valores de (col) y (val)
            question = Question(col, val)
            # Recuperamos los arreglos que cumplan o no la pregunta 
            true_rows, false_rows = partition(rows, question) 

            # Inicializamos la variable de la ganancia y la actualizaremos      
            gain = 0
            if len(true_rows) != 0 and len(false_rows) != 0:
                gain = info_gain(true_rows, false_rows, current_uncertainty)
            
            return (col, val, gain)
        
        # Recorremos los valores de la data
        values = unique_vals(rows,col)
        # Convertimos la data en un RDD
        palavrasRDD = sc.parallelize(values, os.cpu_count())
        # Recorremos nuestro RDD con un map, y hallamos su ganancia y question con la funcion
        pluralRDD = palavrasRDD.map(best_question_and_gain)
        # Recuperamos el resultado en un array de tuplas
        result += pluralRDD.collect()
    
    # Hallamos la maxima ganancia del resultado
    max_value = sc.parallelize(result).sortBy(lambda x: x[2]).collect()
    index = len(max_value) - 1
    # Recuperamos la mejor ganancia
    best_gain = max_value[index][2] 
    # Recuperamos los valores de la question, y creamos su clase
    best_question = Question(max_value[index][0],max_value[index][1]) 
        

    return best_gain, best_question

### **Clase Hoja**
---

In [ ]:
class Leaf:
    """Un nodo Hoja clasifica los datos.

     Esto contiene un diccionario de clase (por ejemplo, "Apple") -> número de veces
     aparece en las filas de los datos de entrenamiento que llegan a esta hoja.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [ ]:
def print_leaf(counts):
    """Una forma más agradable de imprimir las predicciones en una hoja."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

### **Clase nodo de decisión**
---

In [ ]:
class Decision_Node:
    """Un nodo de decisión hace una pregunta.

     Esto contiene una referencia a la pregunta y a los dos nodos secundarios.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

### **Construcción del árbol**
---

In [ ]:
def build_tree(rows):
    """Construcción del árbol.
    """
    # Intente particionar el conjunto de datos en cada uno de los atributos únicos,
    # calcular la ganancia de información, y devuelve la pregunta que produce la mayor ganancia.
    
    gain, question = find_best_split(rows)

    # Caso base: no se obtiene más información
    # Ya que no podemos hacer más preguntas,
    # devolveremos una hoja.
    if gain == 0:
        return Leaf(rows)

    # Si llegamos aquí, hemos encontrado una característica / valor útil
    # para particionar.
    true_rows, false_rows = partition(rows, question)

    # Construir recursivamente la rama verdadera.
    true_branch = build_tree(true_rows)

    # Construir recursivamente la rama falsa.
    false_branch = build_tree(false_rows)

    # Devolver un nodo Pregunta.
    # Esto registra la mejor característica/valor para preguntar en este punto,
    # así como las ramas a seguir
    # dependiendo de la respuesta.
    return Decision_Node(question, true_branch, false_branch)

### **Impresión del Árbol**
---

In [ ]:
def print_tree(node, spacing=""):
    """La función de impresión de árboles más elegante del mundo."""

    # Caso base: hemos llegado a una hoja
    if isinstance(node, Leaf):
        print (spacing + "Prediccion", node.predictions)
        return

    # Imprimir la pregunta en este nodo
    print (spacing + str(node.question))

    # Recursión en la rama verdadera
    print (spacing + '--> Verdadero:')
    print_tree(node.true_branch, spacing + "  ")

    # Recursión en la rama falsa
    print (spacing + '--> Falso:')
    print_tree(node.false_branch, spacing + "  ")

### **Clasificación**
---

In [ ]:
def classify(row, node):
    # Caso base: hemos llegado a una hoja
    if isinstance(node, Leaf):
        return node.predictions

    # Decide si seguir la rama verdadera o la rama falsa.
    # Comparar la característica/valor almacenado en el nodo,
    # al ejemplo que estamos considerando.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

### **Ejemplo 1**
---

In [ ]:
# Etiquetas de columna.
# Estos se usan solo para imprimir el árbol.
header = ["color", "diametro", "etiqueta"]

In [ ]:
training_data = [
    ['Verde', 3, 'Manzana'],
    ['Amarillo', 3, 'Manzana'],
    ['Rojo', 1, 'Uva'],
    ['Rojo', 1, 'Uva'],
    ['Amarillo', 3, 'Limon'],
]

In [ ]:
my_tree = build_tree(training_data)

In [ ]:
print_tree(my_tree)

Es diametro >= 3?
--> Verdadero:
  Es color == Amarillo?
  --> Verdadero:
    Prediccion {'Manzana': 1, 'Limon': 1}
  --> Falso:
    Prediccion {'Manzana': 1}
--> Falso:
  Prediccion {'Uva': 2}




<center/>
<a href="https://ibb.co/4jgkQ8v"><img src="https://i.ibb.co/19JcSry/Diagrama-en-blanco-P-gina-1-5.png" alt="Diagrama-en-blanco-P-gina-1-5" border="0" width=500px></a>
</center>

### **Ejemplo 1 - Test**
---

In [ ]:
testing_data = [
    ['Verde', 3, 'Manzana'],
    ['Amarillo', 3, 'Manzana'],
    ['Rojo', 1, 'Uva'],
    ['Rojo', 1, 'Uva'],
    ['Amarillo', 3, 'Limon'],
]

In [ ]:
for row in testing_data:
    print ("Actual: %s. Prediccion: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: Manzana. Prediccion: {'Manzana': '100%'}
Actual: Manzana. Prediccion: {'Manzana': '50%', 'Limon': '50%'}
Actual: Uva. Prediccion: {'Uva': '100%'}
Actual: Uva. Prediccion: {'Uva': '100%'}
Actual: Limon. Prediccion: {'Manzana': '50%', 'Limon': '50%'}


### **Ejemplo 2**
---

In [ ]:
import pandas as pd
df = pd.read_csv("iris.csv")
header = list(df.columns)
FlowerSet=df.to_numpy()
print(header)


['sepal.length', 'sepal.width', 'petal.length', 'petal.width', 'variety']


In [ ]:
my_tree2 = build_tree(FlowerSet)

In [ ]:
print_tree(my_tree2)

Es petal.width >= 1.0?
--> Verdadero:
  Es petal.width >= 1.8?
  --> Verdadero:
    Es petal.length >= 4.9?
    --> Verdadero:
      Prediccion {'Virginica': 43}
    --> Falso:
      Es sepal.width >= 3.2?
      --> Verdadero:
        Prediccion {'Versicolor': 1}
      --> Falso:
        Prediccion {'Virginica': 2}
  --> Falso:
    Es petal.length >= 5.0?
    --> Verdadero:
      Es petal.width >= 1.6?
      --> Verdadero:
        Es petal.length >= 5.8?
        --> Verdadero:
          Prediccion {'Virginica': 1}
        --> Falso:
          Prediccion {'Versicolor': 2}
      --> Falso:
        Prediccion {'Virginica': 3}
    --> Falso:
      Es petal.width >= 1.7?
      --> Verdadero:
        Prediccion {'Virginica': 1}
      --> Falso:
        Prediccion {'Versicolor': 47}
--> Falso:
  Prediccion {'Setosa': 50}


In [ ]:
for row in FlowerSet:
    print ("Actual: %s. Prediccion: %s" %
           (row[-1], print_leaf(classify(row, my_tree2))))

Actual: Setosa. Prediccion: {'Setosa': '100%'}
Actual: Setosa. Prediccion: {'Setosa': '100%'}
Actual: Setosa. Prediccion: {'Setosa': '100%'}
Actual: Setosa. Prediccion: {'Setosa': '100%'}
Actual: Setosa. Prediccion: {'Setosa': '100%'}
Actual: Setosa. Prediccion: {'Setosa': '100%'}
Actual: Setosa. Prediccion: {'Setosa': '100%'}
Actual: Setosa. Prediccion: {'Setosa': '100%'}
Actual: Setosa. Prediccion: {'Setosa': '100%'}
Actual: Setosa. Prediccion: {'Setosa': '100%'}
Actual: Setosa. Prediccion: {'Setosa': '100%'}
Actual: Setosa. Prediccion: {'Setosa': '100%'}
Actual: Setosa. Prediccion: {'Setosa': '100%'}
Actual: Setosa. Prediccion: {'Setosa': '100%'}
Actual: Setosa. Prediccion: {'Setosa': '100%'}
Actual: Setosa. Prediccion: {'Setosa': '100%'}
Actual: Setosa. Prediccion: {'Setosa': '100%'}
Actual: Setosa. Prediccion: {'Setosa': '100%'}
Actual: Setosa. Prediccion: {'Setosa': '100%'}
Actual: Setosa. Prediccion: {'Setosa': '100%'}
Actual: Setosa. Prediccion: {'Setosa': '100%'}
Actual: Setos

### **Predicción**
---

In [ ]:
print_leaf(classify([5.1,3.5,1.4,0.2,0], my_tree2))

{'Setosa': '100%'}